In [1]:
from keras.layers import Input, Lambda, Dense, Flatten,GlobalAveragePooling2D
from keras.models import Model
from keras.applications.resnet import ResNet152
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [3]:
train_path = 'E:\Deep Learning\Flower Recognization/processed_data/train'
valid_path = 'E:\Deep Learning\Flower Recognization/processed_data/val'
test_path = 'E:\Deep Learning\Flower Recognization/processed_data/test'

In [4]:
res = ResNet152(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# don't train existing weights
for layer in res.layers:
  layer.trainable = False

In [6]:
 # useful for getting number of classes
folders = glob('E:\Deep Learning\Flower Recognization/processed_data/train/*')
len(folders)

5

In [7]:
x = Flatten()(res.output)
x = Dense(1024, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=res.input, outputs=prediction)

In [8]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 230, 230, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 56, 56, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 161,137,541 (614.69 MB)

 Trainable params: 102,766,597 (392.02 MB)

 Non-trainable params: 58,370,944 (222.67 MB)

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

val_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2588 images belonging to 5 classes.
Found 869 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [11]:
op = model.fit(
  training_set,
  epochs=50,
  validation_data=val_set
)

Epoch 1/50


C:\Users\Aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


81/81 ━━━━━━━━━━━━━━━━━━━━ 247s 3s/step - accuracy: 0.2619 - loss: 19.5126 - val_accuracy: 0.3047 - val_loss: 1.7868
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 231s 3s/step - accuracy: 0.3566 - loss: 1.9908 - val_accuracy: 0.3012 - val_loss: 2.9136
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 231s 3s/step - accuracy: 0.3921 - loss: 2.0254 - val_accuracy: 0.4337 - val_loss: 1.8884
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 231s 3s/step - accuracy: 0.4430 - loss: 1.6388 - val_accuracy: 0.3965 - val_loss: 1.9400
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 232s 3s/step - accuracy: 0.4581 - loss: 1.5308 - val_accuracy: 0.4640 - val_loss: 1.4746
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 233s 3s/step - accuracy: 0.5015 - loss: 1.2799 - val_accuracy: 0.4756 - val_loss: 1.3941
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 232s 3s/step - accuracy: 0.5227 - loss: 1.2480 - val_accuracy: 0.4209 - val_loss: 1.5867
Epoch 8/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 232s 3s/step - accuracy: 0.5336 - loss: 1.2665 - val_accuracy: 0.4663 - val_loss: 1

In [12]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")


28/28 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.5994 - loss: 1.0772
Test loss: 1.1353368759155273
Test accuracy: 0.5765247344970703
